In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [63]:
import numpy as np
from sklearn.metrics import root_mean_squared_error


class MiniBatchGradientDescend:
    def __init__(self, lr, max_iter, tol, batch_size):
        self.lr = lr  # learning rate
        self.max_iter = max_iter
        self.tol = tol
        self.weights_ = None
        self.intercept_ = None
        self.batch_size = batch_size
        self.lossi = []

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n = X.shape[0]

        X = np.c_[np.ones(n), X]
        n_features = X.shape[1]

        # initialize weights
        weights = np.random.normal(size=n_features)/100

        converged = False
        m = self.batch_size

        for epoch in range(self.max_iter):

            # ---- OPTIONAL but recommended: shuffle each epoch ----
            perm = np.random.permutation(n)
            X, y = X[perm], y[perm]

            for j in range(n // m):
                # FIXED slicing (no -1)
                X_batch = X[j*m:(j+1)*m]
                y_batch = y[j*m:(j+1)*m]

                y_pred = X_batch @ weights
                e = y_batch - y_pred

                loss = root_mean_squared_error(y_batch, y_pred)
                self.lossi.append(loss)

                # FIXED gradient scaling
                coef_grad = (-2 / len(y_batch)) * X_batch.T @ e
                print(coef_grad)
                # FIXED convergence check
                if np.linalg.norm(coef_grad) < self.tol:
                    converged = True
                    break

                weights = weights - self.lr * coef_grad

            # FIXED: stop outer loop too
            if converged:
                break

        if not converged:
            print(f"Warning: Did not converge after {self.max_iter} steps.")

        self.weights_ = weights[1:]
        self.intercept_ = weights[0]
        self.step = epoch
        return self

    def predict(self, X):
        X = np.array(X)
        return X @ self.weights_ + self.intercept_

    def score(self, X, y):
        y_pred = self.predict(X)
        mse = np.mean((y - y_pred) ** 2)
        return 1 - mse / np.var(y)  # R^2 score


mb = MiniBatchGradientDescend(lr=0.000, max_iter=15, tol=0.1, batch_size=5)


In [65]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df2 = pd.read_csv('advertising.csv')

X1 = df2.iloc[:, :-1]
y1 = df2.iloc[:, -1]
X1_scaled = pd.DataFrame(sc.fit_transform(X1), columns=X1.columns)

mb.fit(X1_scaled, y1)

[-34.90362329 -23.13021904 -20.82976256  -6.2152874 ]
[-27.7307167    2.71876683 -12.41105344   6.75229645]
[-35.98455125 -16.09098109 -35.66796196 -19.9600655 ]
[-31.46935577 -17.71237266 -12.00638833 -25.12545101]
[-30.90842159  -1.83096159 -11.23075676 -12.05866228]
[-30.43395885 -18.67015242  18.16101912  14.53438387]
[-30.14482185  -4.51063786  -7.06069508  -3.14944849]
[-26.77402964   0.49152558  11.95800192  21.71602337]
[-28.95075483   5.45031012   0.58438413  -5.761502  ]
[-26.83292813  -3.30019792  12.82140071   1.15293041]
[-34.46098361 -18.51870347  20.02554016  27.65070946]
[-25.6828065    7.70717127  27.19623006  -1.91999306]
[-27.98868885   2.69355618  12.60960707   5.20728988]
[-27.66308142   5.47251008  -6.30796223  -2.02207202]
[-27.18583751  -3.58924919  14.07990419   9.62975699]
[-25.33196839   5.61819043 -16.42997035   6.84670377]
[-33.37529095  -8.36903038 -23.46470409   5.37636227]
[-34.91026547 -33.84980702 -14.72844373  -0.09583918]
[-33.96548397 -11.48043101 -

In [56]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("scaler", StandardScaler()),
    ("sgd", SGDRegressor(
        loss="squared_error",   # MSE
        learning_rate="constant",
        eta0=0.001,             # learning rate
        max_iter=1000,
        tol=1e-4,
        random_state=42
    ))
])

model.fit(X1_scaled, y1)


,steps,"[('scaler', ...), ('sgd', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,loss,'squared_error'
,penalty,'l2'
,alpha,0.0001
,l1_ratio,0.15


In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, r2_score
r2_score(model.predict(X1_scaled), y1)

0.8920132630031872